<a href="https://colab.research.google.com/github/alexlautw9527/for_notebook/blob/main/%E7%8E%89%E5%B1%B12021%E5%86%AC%E5%AD%A3%E8%B3%BD%E6%8E%A8%E8%96%A6%E7%B3%BB%E7%B5%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd 
import numpy as np
import dask.dataframe as dd


In [38]:
import tensorflow as tf


In [2]:
%%capture
!python -m pip install "dask[complete]" 

In [53]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(16, activation='softmax')
])

In [63]:
model.compile(optimizer='rmsprop',
       loss='mae')

In [64]:
train_y_org = train_y_org.fillna(0)
train_df = train_df.loc[train_int_id].fillna(0)


In [65]:
model.fit(train_df.loc[train_int_id].values, train_y.loc[train_int_id].values, batch_size=1, epochs=100)


Epoch 1/100
  2992/371201 [..............................] - ETA: 11:35 - loss: nan

KeyboardInterrupt: ignored

In [3]:
## 消費類別在這些list才是預測範圍

pred_list = ['2',
'6',
'10',
'12',
'13',
'15',
'18',
'19',
'21',
'22',
'25',
'26',
'36',
'37',
'39',
'48']

In [7]:
'''
!cp /content/drive/MyDrive/02_GNL_DATA/2021玉山人工智慧公開挑戰賽冬季賽訓練資料集.gz.gz /content
!gunzip /content/2021玉山人工智慧公開挑戰賽冬季賽訓練資料集.gz.gz
!tar -xvf /content/2021玉山人工智慧公開挑戰賽冬季賽訓練資料集.gz

import dask.dataframe as dd
df = dd.read_csv('/content/tbrain_cc_training_48tags_hash_final.csv',assume_missing=True)

%%time
## 把csv切成32個parquet格式

df = df.set_index('chid')
df = df.map_partitions(lambda df: df.sort_values(['chid', 'dt']))

df = df.repartition(npartitions=32) 
df = dd.to_parquet(df,path='/content/data.parquet', engine='pyarrow')

!zip -r esun2021winterdata_parquet.zip /content/data.parquet
'''

In [6]:
## 載入切好格式

df = dd.read_parquet('/content/data.parquet', engine='pyarrow')


In [ ]:
%%time
##看看一個id都幾筆資料
df.groupby('chid').size().describe().compute()

CPU times: user 38.6 s, sys: 18.1 s, total: 56.7 s
Wall time: 18.2 s


count    500000.000000
mean         65.951306
std          55.929864
min           1.000000
25%          24.000000
50%          50.000000
75%          93.000000
max         441.000000
dtype: float64

In [ ]:
train_org

In [7]:
train_org = df[df['dt']<11]
train_y_org = df[df['dt']==11]

In [26]:
train_df = train_org.groupby(['chid','shop_tag'])['txn_amt'].sum().compute().unstack(fill_value=0)

In [27]:
train_df = train_df.merge(profile_df, how='left', on='chid')

In [32]:
train_int_id = list(set(train_df.index)&set(train_y_org.index))
train_int_id.sort()

,1,10,11,12,13,14,15,16,17,18,19,2,20,21,22,23,24,25,26,27,28,29,3,30,31,32,33,34,35,36,37,38,39,4,40,41,42,43,44,45,46,47,48,5,6,7,8,9,other,masts,educd,trdtp,naty,poscd,cuorg,gender_code,age
chid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10000000.0,0.0,95253.220275,0.000000,40382.699592,0.000000,0.000000,4684.483372,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,4235.791755,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,7604.385348,41169.198215,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,6800.279121,2.0,2.0,2.0,1.0,3.0,30.0,1.0,4.0
10000001.0,0.0,0.000000,0.000000,18895.258071,0.000000,0.000000,30321.321469,0.000000,0.000000,8019.906666,0.000000,10421.017869,0.0,5956.549195,0.000000,0.0,0.0,0.000000,14312.809660,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,29002.204986,0.0,12791.090263,12542.937984,0.0,16480.143886,0.0,10041.144817,0.000000,0.0,0.0,0.000000,37872.347043,0.000000,0.000000,31548.344182,0.0,16412.851077,0.000000,0.0,0.0,28099.577869,2.0,4.0,11.0,1.0,3.0,30.0,0.0,5.0
10000004.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6037.642859,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,6308.563642,0.0,0.0,0.000000,0.0,11546.072969,26001.789010,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,16767.271182,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,2.0,5.0,15.0,1.0,99.0,30.0,1.0,2.0
10000006.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,33925.711654,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,8200.950959,2.0,6.0,15.0,1.0,99.0,30.0,0.0,7.0
10000008.0,0.0,37895.584044,0.000000,16177.921723,0.000000,7990.412128,61046.916735,0.000000,0.000000,30912.804987,36043.884637,29259.027953,0.0,0.000000,0.000000,0.0,0.0,18790.253661,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,27505.220900,0.0,0.000000,144593.916374,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,122135.945768,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,17341.904355,1.0,5.0,2.0,1.0,99.0,30.0,0.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10499992.0,0.0,47838.121052,6257.629931,0.000000,10322.373189,0.000000,15905.507206,0.000000,0.000000,0.000000,6674.712292,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,32528.540359,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,12759.396271,75823.075348,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,15806.366368,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,31482.379649,1.0,5.0,7.0,1.0,2.0,30.0,0.0,3.0
10499993.0,0.0,44332.291433,0.000000,22976.876231,0.000000,0.000000,44365.688937,0.000000,5874.031043,7560.706118,0.000000,9442.936693,0.0,0.000000,4335.715997,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,3149.494657,3828.277207,0.000000,0.0,0.0,6598.815718,0.0,8695.277775,22205.776431,0.0,7945.887166,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,9131.600003,0.0,8019.906666,8019.906666,0.0,0.0,36950.854609,2.0,3.0,5.0,1.0,99.0,30.0,0.0,3.0
10499996.0,0.0,0.000000,7016.248056,0.000000,0.000000,0.000000,23444.225443,11556.891814,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,5773.036485,5773.036485,0.0,0.000000,0.0,0.000000,21156.747131,0.0,0.0,0.000000,17373.977169,0.000000,9794.580217,50459.860651,0.0,22041.208357,0.000000,0.0,0.0,5041.159508,2.0,3.0,11.0,1.0,2.0,30.0,1.0,3.0


In [9]:
%%time
train_y_org = train_y_org.groupby(['chid','shop_tag'])['txn_amt'].sum().compute().unstack(fill_value=0)

CPU times: user 32 s, sys: 21.2 s, total: 53.2 s
Wall time: 18 s


In [10]:
train_y_org= train_y_org[pred_list]

In [61]:
train_y = train_y_org.div(train_y_org.sum(axis=1), axis=0).fillna(0)

In [19]:
train_y.shape

(378500, 16)

In [ ]:
train_org

In [50]:
test_org

,dt,shop_tag,txn_cnt,txn_amt,domestic_offline_cnt,domestic_online_cnt,overseas_offline_cnt,overseas_online_cnt,domestic_offline_amt_pct,domestic_online_amt_pct,overseas_offline_amt_pct,overseas_online_amt_pct,card_1_txn_cnt,card_2_txn_cnt,card_3_txn_cnt,card_4_txn_cnt,card_5_txn_cnt,card_6_txn_cnt,card_7_txn_cnt,card_8_txn_cnt,card_9_txn_cnt,card_10_txn_cnt,card_11_txn_cnt,card_12_txn_cnt,card_13_txn_cnt,card_14_txn_cnt,card_other_txn_cnt,card_1_txn_amt_pct,card_2_txn_amt_pct,card_3_txn_amt_pct,card_4_txn_amt_pct,card_5_txn_amt_pct,card_6_txn_amt_pct,card_7_txn_amt_pct,card_8_txn_amt_pct,card_9_txn_amt_pct,card_10_txn_amt_pct,card_11_txn_amt_pct,card_12_txn_amt_pct,card_13_txn_amt_pct,card_14_txn_amt_pct,card_other_txn_amt_pct,masts,educd,trdtp,naty,poscd,cuorg,slam,gender_code,age,primary_card
npartitions=32,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10000000.0,float64,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
10014869.0,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10485007.0,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10499999.0,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [57]:
test = test_org.loc[[10000000, 10000001]].compute()

In [58]:
test.groupby('chid').apply

,dt,shop_tag,txn_cnt,txn_amt,domestic_offline_cnt,domestic_online_cnt,overseas_offline_cnt,overseas_online_cnt,domestic_offline_amt_pct,domestic_online_amt_pct,overseas_offline_amt_pct,overseas_online_amt_pct,card_1_txn_cnt,card_2_txn_cnt,card_3_txn_cnt,card_4_txn_cnt,card_5_txn_cnt,card_6_txn_cnt,card_7_txn_cnt,card_8_txn_cnt,card_9_txn_cnt,card_10_txn_cnt,card_11_txn_cnt,card_12_txn_cnt,card_13_txn_cnt,card_14_txn_cnt,card_other_txn_cnt,card_1_txn_amt_pct,card_2_txn_amt_pct,card_3_txn_amt_pct,card_4_txn_amt_pct,card_5_txn_amt_pct,card_6_txn_amt_pct,card_7_txn_amt_pct,card_8_txn_amt_pct,card_9_txn_amt_pct,card_10_txn_amt_pct,card_11_txn_amt_pct,card_12_txn_amt_pct,card_13_txn_amt_pct,card_14_txn_amt_pct,card_other_txn_amt_pct,masts,educd,trdtp,naty,poscd,cuorg,slam,gender_code,age,primary_card
chid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10000000.0,12.0,45,1.0,7116.667890,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,2.0,1.0,3.0,30.0,79557.742574,1.0,4.0,1.0
10000000.0,13.0,10,2.0,9259.128624,0.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,2.0,1.0,3.0,30.0,79557.742574,1.0,4.0,1.0
10000000.0,13.0,other,1.0,6791.082525,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,2.0,1.0,3.0,30.0,79557.742574,1.0,4.0,1.0
10000000.0,14.0,12,2.0,10507.131178,0.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,2.0,1.0,3.0,30.0,79557.742574,1.0,4.0,1.0
10000000.0,14.0,10,4.0,12458.712002,0.0,4.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,2.0,1.0,3.0,30.0,79557.742574,1.0,4.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10000001.0,23.0,10,1.0,12847.384866,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,4.0,11.0,1.0,3.0,30.0,62730.806297,0.0,5.0,1.0
10000001.0,24.0,48,2.0,7060.142894,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,4.0,11.0,1.0,3.0,30.0,62730.806297,0.0,5.0,1.0
10000001.0,24.0,45,2.0,8455.731170,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,4.0,11.0,1.0,3.0,30.0,62730.806297,0.0,5.0,1.0


In [22]:
profile_df = df.groupby('chid').last().compute()

In [28]:
profile_df.columns

Index(['dt', 'shop_tag', 'txn_cnt', 'txn_amt', 'domestic_offline_cnt',
       'domestic_online_cnt', 'overseas_offline_cnt', 'overseas_online_cnt',
       'domestic_offline_amt_pct', 'domestic_online_amt_pct',
       'overseas_offline_amt_pct', 'overseas_online_amt_pct', 'card_1_txn_cnt',
       'card_2_txn_cnt', 'card_3_txn_cnt', 'card_4_txn_cnt', 'card_5_txn_cnt',
       'card_6_txn_cnt', 'card_7_txn_cnt', 'card_8_txn_cnt', 'card_9_txn_cnt',
       'card_10_txn_cnt', 'card_11_txn_cnt', 'card_12_txn_cnt',
       'card_13_txn_cnt', 'card_14_txn_cnt', 'card_other_txn_cnt',
       'card_1_txn_amt_pct', 'card_2_txn_amt_pct', 'card_3_txn_amt_pct',
       'card_4_txn_amt_pct', 'card_5_txn_amt_pct', 'card_6_txn_amt_pct',
       'card_7_txn_amt_pct', 'card_8_txn_amt_pct', 'card_9_txn_amt_pct',
       'card_10_txn_amt_pct', 'card_11_txn_amt_pct', 'card_12_txn_amt_pct',
       'card_13_txn_amt_pct', 'card_14_txn_amt_pct', 'card_other_txn_amt_pct',
       'masts', 'educd', 'trdtp', 'naty', 'posc

In [25]:
profile_df = profile_df[[
  'masts', 'educd', 'trdtp', 'naty', 'poscd', 
  'cuorg','gender_code', 'age']]

In [34]:
profile_df['masts'].value_counts()

1.0    254529
2.0    245094
3.0       374
Name: masts, dtype: int64

In [35]:
profile_df['educd'].value_counts()

3.0    192602
5.0     97614
4.0     87598
6.0     63800
2.0     53533
1.0      4850
Name: educd, dtype: int64

In [21]:
%%time
df['dt'].unique().compute()

CPU times: user 1.44 s, sys: 557 ms, total: 2 s
Wall time: 821 ms


0      1.0
1      2.0
2      3.0
3      4.0
4      5.0
5      6.0
6      7.0
7      8.0
8      9.0
9     10.0
10    11.0
11    12.0
12    13.0
13    14.0
14    15.0
15    16.0
16    17.0
17    18.0
18    19.0
19    20.0
20    21.0
21    22.0
22    23.0
23    24.0
Name: dt, dtype: float64

In [ ]:
%%time

## 計算by id各類別總計消費金額
newdf = df.groupby(['chid','shop_tag'])['txn_amt'].sum().compute().unstack(fill_value=0)

CPU times: user 1min 11s, sys: 20.4 s, total: 1min 32s
Wall time: 34.9 s


In [ ]:
newdf.head()

shop_tag,1,10,11,12,13,14,15,16,17,18,19,2,20,21,22,23,24,25,26,27,28,29,3,30,31,32,33,34,35,36,37,38,39,4,40,41,42,43,44,45,46,47,48,5,6,7,8,9,other
chid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10000000.0,0.0,191770.225706,0.0,76268.969509,0.0,0.0,4684.483372,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,4235.791755,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,62218.660703,81337.066461,0.0,9156.086033,0.0,0.00000,0.0,0.0,0.000000,40746.482820
10000001.0,0.0,12847.384866,0.0,44104.144043,0.0,0.0,57821.590353,12643.832959,0.000000,8019.906666,0.0,17649.573492,0.0,25146.943067,3945.661088,0.0,0.0,0.0,39478.512075,0.0,0.0,0.0,6308.563642,0.0,595.018412,0.0,0.0,51329.246565,0.0,26998.210065,56105.751640,0.0,49331.267298,0.0,18721.636466,0.0,14317.563657,0.0,6048.351042,63374.858212,0.000000,0.0,84246.368876,0.0,26023.22562,0.0,0.0,0.000000,50488.365846
10000002.0,0.0,16755.695323,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,3571.095195,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,5692.923612,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.00000,0.0,0.0,0.000000,37649.183244
10000003.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,8554.030804,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,8023.582985,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.00000,0.0,0.0,0.000000,0.000000
10000004.0,0.0,0.000000,0.0,15271.878083,0.0,0.0,5773.036485,0.000000,6037.642859,0.000000,0.0,10825.822388,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,12617.127283,0.0,0.0,0.000000,0.0,11546.072969,34238.290641,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,19790.314068,0.000000,0.0,5419.225995,0.0,0.00000,0.0,0.0,10659.532107,12386.980405


In [ ]:
np.argmax(newdf,axis=1)

array([ 1, 42, 48, ..., 35, 39, 14])

In [ ]:

## 取topn最大值的位置 (為了直接映射colname, 而colname即是類別)
n=0
(-newdf.values).argsort(axis=1)[:,n]

array([ 1, 42, 48, ..., 35, 39, 14])

In [ ]:
top1 = [list(newdf.columns)[x] for x in (-newdf.values).argsort(axis=1)[:,0]]
top2 = [list(newdf.columns)[x] for x in (-newdf.values).argsort(axis=1)[:,1]]
top3 = [list(newdf.columns)[x] for x in (-newdf.values).argsort(axis=1)[:,2]]

In [ ]:
top_df = pd.DataFrame({'chid':newdf.index, 'top1':top1, 'top2':top2, 'top3':top3})
top_df['chid'] = top_df['chid'].astype(int)

In [ ]:
top_df['top1'] = np.where(top_df['top1'].isin(pred_list), top_df['top1'], np.nan)
top_df['top2'] = np.where(top_df['top2'].isin(pred_list), top_df['top2'], np.nan)
top_df['top3'] = np.where(top_df['top3'].isin(pred_list), top_df['top3'], np.nan)

37 15 36為先驗top3最多的消費類別

In [ ]:
top_df = top_df.fillna('37')

In [ ]:
top_df.to_csv('submit2.csv', index=False)

In [ ]:
top_df

,chid,top1,top2,top3
0,10000000,10,37,12
1,10000001,48,37,15
2,10000002,37,10,37
3,10000003,22,37,37
4,10000004,37,37,12
...,...,...,...,...
499995,10499995,2,10,37
499996,10499996,48,15,37
499997,10499997,37,25,37
499998,10499998,37,39,37
